## Feature Development Extraction: Neurophysiology [dyskinesia project]




<b> Content </b>


<b> Aperiodic estimates </b>
Relevant literature:
- Periodic and a-periodic components relevance and interaction, different reasons (per + a-per) for signal changes observed within a specific bandwidth. Aperiodic component (complicated) vs exponent (1/f) (Donoghue, ..., Shestyuk & Voytek, Nature Neurosc 2020 : https://www.nature.com/articles/s41593-020-00744-x)
- cycle-by-cycle features: bycycle toolbox (Cole & Voytek, J of Neurophys 2019, https://journals.physiology.org/doi/full/10.1152/jn.00273.2019)
- aperiodic component, PD severity, and cortico-subcortico-activity, Bush & Zou, Richardson, bioRxiv 2023 https://www.biorxiv.org/content/10.1101/2023.02.08.527719v1?rss=1

<b> Periodic component analysis: </b> 
- Try Wavelet Dceomposition vs Welch (tapered) Spectral Decomposition





### 0. Loading packages and functions, defining paths



In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import pandas as pd
import numpy as np
import sklearn as sk
from itertools import product, compress

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version



In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)
print('matplotlib', plt_version)
## FEB 2022:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')

In [ ]:
os.chdir(codepath)
# own utility functions
import utils.utils_fileManagement as utilsFiles
import utils.utils_windowing as utilsWindows
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)

# own data exploration functions
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.ft_processing_helpers as ftProc

import lfpecog_plotting.plotHelpers as plotHelp

### Plot ssd'd Spectral Descriptives

#### SSD time freq plotting


In [ ]:
import lfpecog_plotting.plot_timeFreqs_ssd_psds as plot_ssd_TFs

In [ ]:
WIN_LEN=10
WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
SSD_BROAD=True
INCL_STN_ONLY_PTS=False
IGNORE_PTS=['011', '104', '106']

In [ ]:
importlib.reload(ftProc)
importlib.reload(importClin)
importlib.reload(plot_ssd_TFs)

# CONVERT ONLY STNS!!!!!
FIG_DATE = '0202b'
SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)

for sub in SUBS:
    if sub not in ['008',]:
        print(f'skip {sub}')
        continue
    
    print(f'PLOT SUB-{sub}')
    plot_ssd_TFs.plot_indiv_ssd_timefreq_allSources(
        sub, fig_name_base=f'{FIG_DATE}_ssdTimeFreq',
        FT_VERSION=FT_VERSION,
        DATA_VERSION=DATA_VERSION,
        ZSCORE=False, LOG_POWER=True,
        SAVE_PLOT=True,
        FOR_FIG=True,)
    # except:
    #     print(f'sub {sub} error')

#### SSD PSD plotting

In [ ]:

import lfpecog_analysis.get_SSD_timefreqs as ssd_TimeFreq
# import lfpecog_analysis.process_connectivity as procConn
import lfpecog_analysis.get_acc_task_derivs as accDerivs
import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_PSDs
import lfpecog_plotting.plot_COH_spectra as plot_COHs
import lfpecog_analysis.psd_analysis_classes as psdAnlClass

In [ ]:
# WIN_LEN=10
# WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
# SSD_BROAD=True
# INCL_STN_ONLY_PTS=True
IGNORE_PTS=['011', '104', '106']

In [ ]:
SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)
print(len(SUBS))

In [ ]:
# importlib.reload(procConn)

importlib.reload(ssd_TimeFreq)


TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
    SUBS=SUBS, FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
    GET_CONNECTIVITY=False,  # 'trgc'
)


Plot 10-second epochs using new (Feb-Mar 24) plotting scripts

In [ ]:
print(f"shape of .values array: {TFs['012']['lfp_left'].values.shape}")
print(f"shape of .freqs array: {TFs['012']['lfp_left'].freqs.shape}")
print(f"shape of .times array: {TFs['012']['lfp_left'].times.shape}")

In [ ]:
TFs['012']['lfp_left'][2]

In [ ]:
BLs = psdAnlClass.get_selectedEphys(
    FEATURE='POWER',
    STATE_SEL='baseline',
    MIN_SEL_LENGTH=10,
    LOAD_PICKLE=True,
    USE_EXT_HD=True,
    PREVENT_NEW_CREATION=False,
    RETURN_PSD_1sec=False,
    verbose=False,
)

In [ ]:
def get_unilat_lid_timesscores():

    unilid_subs = {}

    for sub in TFs.keys():
        unilid = {}
        scores_l = importClin.get_cdrs_specific(
            sub=sub, INCL_CORE_CDRS=False, side='left'
        )[1]
        scores_r = importClin.get_cdrs_specific(
            sub=sub, INCL_CORE_CDRS=False, side='right'
        )[1]
        l_lid = np.logical_and(scores_l > 0, scores_r == 0)
        r_lid = np.logical_and(scores_r > 0, scores_l == 0)
    
        if any(l_lid):
            lidside = 'left'
            otherscores = scores_r
        elif any(r_lid):
            lidside = 'right'
            otherscores = scores_l
        else: continue

        ts, lids = importClin.get_cdrs_specific(
            sub=sub, INCL_CORE_CDRS=True, side=lidside
        )
        unilid_subs[f'{sub}_{lidside}'] = (ts, lids, otherscores)

    return unilid_subs

In [ ]:
def unpack_dict_lists(psd_arrs, sub_arrs):
    """
    unpack every lid-category dict-content
    containing of lists of lists/arrays
    """
    for cat_key in psd_arrs:
        psd_arrs[cat_key] = np.array(
            [row for arr in psd_arrs[cat_key] for row in arr]
        )
        sub_arrs[cat_key] = np.array(
            [s for l in sub_arrs[cat_key] for s in l]
        )
        assert len(sub_arrs[cat_key]) == psd_arrs[cat_key].shape[0], (
            f'MISMAtCH in {cat_key}: n-subs {len(sub_arrs[cat_key])} with'
            f' n-rows: {psd_arrs[cat_key].shape}'
        )

    return psd_arrs, sub_arrs

In [ ]:
def prep_10sPSDs_lidCategs(
    SOURCE, TFs, BASELINE,
    IPSI_CONTRA_UNILID: bool = False,
    unilid_subs=None,
):
    """
    
    Parameters:
        - SOURCE: either lfp or ecog
        - TFs: dicts with combined SSDd PSDs
        - BASELINE: class with baseline PSDs
    """
    sources = ['lfp_left', 'lfp_right', 'ecog']
    lid_states = ['nolidbelow30', 'nolidover30', 'nolid',
                'mildlid', 'moderatelid', 'severelid']
    lid_categ_ranges = {'nolid': [0, 0],
                        'mildlid': [1, 3],
                        'moderatelid': [4, 7],
                        'severelid': [8, 20]}

    psd_arrs = {l: [] for l in lid_states}
    sub_arrs = {l: [] for l in lid_states}

    if IPSI_CONTRA_UNILID:
        psd_arrs = {f"{s}_{l}": [] for s, l in product(
            ['contra', 'ipsi'], lid_states[-3:]
        )}
        sub_arrs = {f"{s}_{l}": [] for s, l in product(
            ['contra', 'ipsi'], lid_states[-3:]
        )}

    #ä loop over selected uni-LID subjects and sources
    if not IPSI_CONTRA_UNILID: loop_subs = TFs.keys()
    else: loop_subs = unilid_subs

    for sub_code, src in product(loop_subs, sources):
        if SOURCE not in src: continue

        if IPSI_CONTRA_UNILID: sub, lidside = sub_code.split('_')
        else: sub = sub_code
        if sub.startswith('1') and 'ecog' in src: continue

        if src == 'ecog':  # IPSI_CONTRA_UNILID and 
            ephysside = importClin.get_ecog_side(sub)
            src = f'ecog_{ephysside}'

        # get all ephys value
        psx = TFs[sub][src].values.T
        ps_times = TFs[sub][src].times / 60
        ps_freqs = TFs[sub][src].freqs
        # select relevant freqs
        sel1 = np.logical_and(ps_freqs >= 4, ps_freqs < 35)
        sel2 = np.logical_and(ps_freqs >= 60, ps_freqs < 90)
        f_sel = np.logical_or(sel1, sel2)
        psx = psx[:, f_sel]
        ps_freqs = ps_freqs[f_sel]

        # split values for unilat-LID
        if not IPSI_CONTRA_UNILID:
            lid_times, lid_scores = importClin.get_cdrs_specific(
                sub=sub, INCL_CORE_CDRS=True, side='both'
            )
            # match ephys values with uniLID epochs
            near_time_idx = [np.argmin(abs(lid_times - t))
                             for t in ps_times]
            ps_scores = lid_scores[near_time_idx]
            
        elif IPSI_CONTRA_UNILID:
            # define orientation of hemisphere to unilat-LID
            if 'ecog' not in src:  # ecog ephysside defined above
                ephysside = src.split('_')[1]
            if lidside == ephysside: SIDE = 'ipsi'
            else: SIDE = 'contra'

            lid_times, cdrs, othercdrs = unilid_subs[sub_code]
            unilid_sel = np.logical_and(cdrs > 0, othercdrs == 0)
            uni_times = lid_times[unilid_sel].values
            uni_cdrs = cdrs[unilid_sel].values
            # match ephys values with uniLID epochs
            near_time_idx = [np.argmin(abs(lid_times - t)) for t in ps_times]
            near_times = lid_times[near_time_idx]  # get matching lid times for ephys
            near_cdrs = cdrs[near_time_idx]
            # select value-epochs for uni-LID times
            unilid_sel = np.isin(near_times, uni_times)
            ps_times = ps_times[unilid_sel]
            ps_scores = near_cdrs[unilid_sel]
            psx = psx[unilid_sel, :]
        
        # baseline correct
        try:
            if 'ecog' in src: bl_attr = f'ecog_{sub}_baseline'
            elif 'lfp' in src: bl_attr = f'{src}_{sub}_baseline'
            
            bl = getattr(BASELINE, bl_attr)
            if len(bl.shape) == 2: bl = bl.mean(axis=0)
        except:
            print(f'### WARNING no baseline {src} sub {sub}')
            continue
        psx = ((psx - bl) / bl) * 100

        # select values into LID categories
        for lidkey, cdrs_range in lid_categ_ranges.items():
            cat_sel = np.logical_and(ps_scores >= cdrs_range[0],
                                     ps_scores <= cdrs_range[1])
            if sum(cat_sel) == 0: continue
            cat_psx = psx[cat_sel, :]
            cat_times = ps_times[cat_sel]
            cat_subs = [sub] * sum(cat_sel)
            # add to store dicts
            store_key = lidkey
            if IPSI_CONTRA_UNILID: store_key = f'{SIDE}_{lidkey}'
            psd_arrs[store_key].append(cat_psx)
            sub_arrs[store_key].append(cat_subs)
            print(src, sub_code, lidkey, store_key, cat_psx.shape)

    psd_arrs, sub_arrs = unpack_dict_lists(psd_arrs, sub_arrs)

    return psd_arrs, ps_freqs, sub_arrs


In [ ]:
import lfpecog_plotting.plot_psd_restvsmove as plotRestMovePsds

Plot STN/ECoG Lateralisation during unilat. Dyskinesia

In [ ]:
importlib.reload(plotRestMovePsds)

# SOURCE = 'ecog'   #ecog/lfp
IPSI_CONTRA_UNILID = True
SAVE_PLOT = True
FIG_DATE = '0306'
SHOW_PLOT = False
INCL_STATS = False

# if IPSI_CONTRA_UNILID:
#     unilid_subs = get_unilat_lid_timesscores()
# else:
#     unilid_subs = None

n_rows, n_cols = 1, 2
kw_params = {'sharey': 'row'}

YLIM = (-60, 175)

fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(6*n_cols, 4*n_rows),
                         **kw_params)

for src, ax in zip(['lfp', 'ecog'], axes):

    ### 10-sec PREP has to lead to psd_arrs, psd_freqs, psd_subs
    (psd_arrs,
     ps_freqs,
     sub_arrs) = prep_10sPSDs_lidCategs(
        TFs=TFs, SOURCE=src,
        BASELINE=BLs,
        IPSI_CONTRA_UNILID=IPSI_CONTRA_UNILID,
        unilid_subs=unilid_subs,
    )

    plotRestMovePsds.plot_moveLidSpec_PSDs(
        psd_arrs=psd_arrs,
        psd_freqs=ps_freqs.copy(),
        psd_subs=sub_arrs,
        SOURCE=src,
        PLOT_MOVE_TYPE='unilatLID',
        AX=ax,
        INCL_STATS=INCL_STATS,
    )

y_ax = axes[1].axes.get_yaxis()
ylab = y_ax.get_label()
ylab.set_visible(False)
    
plt.tight_layout()

if SAVE_PLOT:
    FIG_NAME = f'{FIG_DATE}_Lateral_unilatLID_stnecog'
    if INCL_STATS:
        FIG_NAME += '_inclStats'
    FIG_PATH = os.path.join(get_project_path('figures'),
                            'final_Q1_2024',
                            'unilatLID_lateralization')
    
    plt.savefig(os.path.join(FIG_PATH, FIG_NAME),
                dpi=300, facecolor='w',)
    print(f'saved plot {FIG_NAME} in {FIG_PATH}!')

if SHOW_PLOT: plt.show()
else: plt.close()


Plot full 10sec PSDs per LID-category

In [ ]:
### 10-sec PREP has to lead to psd_arrs, psd_freqs, psd_subs
(psd_arrs,
 ps_freqs,
 sub_arrs) = prep_10sPSDs_lidCategs(
    TFs=TFs, SOURCE='ecog',
    BASELINE=BLs,
    IPSI_CONTRA_UNILID=False,
)

In [ ]:
plt.plot(ps_freqs, psd_arrs['nolid'][0])

In [ ]:
sub_arrs['mildlid'].shape

### Plot Coherence Spectra Densities

Plot 4 STN-ECoG Coherencies

In [ ]:
importlib.reload(plot_COHs)

FIG_DATE = '1025'
CUT_MILD_MOD = 4
CUT_MOD_SEV = 8
INCL_CORE_CDRS = False
SAVE_PLOT = True
SHOW_PLOT = False
FILE_NAME = (f'{FIG_DATE}_STNECOG_4_COHs'
            f'_{CUT_MILD_MOD}mod{CUT_MOD_SEV}sev')
if not INCL_CORE_CDRS: FILE_NAME += '_noCore'

fig, axes = plt.subplots(2, 2, figsize=(12, 12),
                         sharey='row')

for i_row, COH_type in enumerate(['sq_coh', 'imag_coh']):
    
    for i_col, MOV in enumerate(['INCL_MOVE', 'EXCL_MOVE']):

        axes[i_row, i_col] = plot_COHs.plot_COH_spectra(
            COH_type=COH_type, COH_source='STN_ECOG',
            SELECT_ON_ACC_RMS=MOV,
            RETURN_AX=True,
            given_ax=axes[i_row, i_col],
            CUT_MILD_MOD=CUT_MILD_MOD,
            CUT_MOD_SEV=CUT_MOD_SEV,
            INCL_CORE_CDRS=INCL_CORE_CDRS,
        )

if SAVE_PLOT:
    path = os.path.join(get_project_path('figures'),
                'ft_exploration',
                f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                'descr_COHs')
    if not os.path.exists(path): os.makedirs(path)
    plt.savefig(os.path.join(path, FILE_NAME),
                facecolor='w', dpi=300,)

if SHOW_PLOT: plt.show()
else: plt.close()

Plot 4 STN-STN Coherencies

In [ ]:
## PLOT STN-STN COHERENCES
importlib.reload(plot_COHs)

SAVE_PLOT = True
SHOW_PLOT = False
FILE_NAME = '1025_STNSTN_4_COHs'

fig, axes = plt.subplots(2, 2, figsize=(12, 12),
                         sharey='row')

for i_row, COH_type in enumerate(['sq_coh', 'imag_coh']):
    
    for i_col, MOV in enumerate(['INCL_MOVE', 'EXCL_MOVE']):

        axes[i_row, i_col] = plot_COHs.plot_COH_spectra(
            COH_type=COH_type, COH_source='STN_STN',
            SELECT_ON_ACC_RMS=MOV,
            RETURN_AX=True,
            given_ax=axes[i_row, i_col]
        )

if SAVE_PLOT:
    path = os.path.join(get_project_path('figures'),
                'ft_exploration',
                f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                'descr_COHs')
    if not os.path.exists(path): os.makedirs(path)
    plt.savefig(os.path.join(path, FILE_NAME),
                facecolor='w', dpi=300,)

if SHOW_PLOT: plt.show()
else: plt.close()


##### Plot Group Results: Spectral changes over Time after L-Dopa, non-LID versus LID

Moved to separate py script (plot_group_metrics_over_time.py)

##### Plot LID-Group Results: Laterality of LID in STN and ECoG

Plot SEVERITY AND LATERALITY OF STN-changes during DYSKINESIA

- LAT_UNI: plot laterality of unilateral LID
- LAT_BILAT: plot laterality of only bilateral LID
- SCALE: combine all uni- and bilateral LID
    - PLOT_ONLY_MATCH == True: plot only STN contralateral to LID
    - PLOT_ONLY_MATCH == False: plot contra-, ipsi-, and bi-lateral STN-LID


In [ ]:
import lfpecog_analysis.psd_lid_stats as psd_stats
import lfpecog_analysis.get_acc_task_derivs as accDerivs

In [ ]:
importlib.reload(ftProc)
importlib.reload(accDerivs)
importlib.reload(importClin)
importlib.reload(plotHelp)
# importlib.reload(psd_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
LAT_or_SCALE = 'LAT_UNI'
STAT_DATE = '1023'
FIG_DATE = '0305'

INCL_PRELID = False
PRELID_MINs=5
SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
SHOW_GAMMA = False
SHOW_SIGN = False
PLOT_ONLY_MATCH = False
SELECT_ACC_RMS= False

fig_name = f'{FIG_DATE}_STN_PSDs_{LAT_or_SCALE}_LID_n{len(SUBS)}'

if INCL_PRELID: fig_name += f'_inclPre{PRELID_MINs}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
# if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'


fsize=14

stn_stats = plot_PSDs.plot_STN_PSD_vs_LID(
    all_timefreqs=TFs,
    CDRS_RATER='Patricia',
    LAT_or_SCALE=LAT_or_SCALE,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    incl_PRE_LID=INCL_PRELID,
    PRELID_MIN=PRELID_MINs,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    RETURN_FREQ_CORR=False,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)


Plot SEVERITY AND LATERALITY OF ECoG-changes during DYSKINESIA

In [ ]:
importlib.reload(accDerivs)
# importlib.reload(psdLID_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
FIG_DATE = '1025'
STAT_DATE = '1023'
LAT_or_SCALE = 'SCALE'
SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS = 8
LOG_POWER = False
BASELINE_CORRECT = True
ZSCORE_FREQS = False
SINGLE_LINE = False
SHOW_GAMMA = False
PLOT_ONLY_MATCH = True
SHOW_SIGN = False
SELECT_ACC_RMS = 'EXCL_MOVE'

fig_name = f'{FIG_DATE}_ECoG_PSDs_LID_{LAT_or_SCALE}_n{len(SUBS)}'

if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'

fsize=14

plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs,
    LAT_or_SCALE=LAT_or_SCALE,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    single_sub_lines=SINGLE_LINE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

Plot LATERALITY during UNILATERAL dyskinesia

In [ ]:
LID_SUBS = [s for s in SUBS if s not in ['101', '109', '017']]
print(LID_SUBS)

In [ ]:
importlib.reload(plot_PSDs)


SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
CDRS_RATER = 'Patricia'

fig_name = f'0305_LATERALITY_STNandECOG_PSDs_unilatLID_n{len(SUBS)}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'



fig, axes = plt.subplots(1, 2, figsize=(12, 6))

fsize=14

plot_PSDs.plot_STN_PSD_vs_LID(
    TFs, sel_subs=LID_SUBS,
    LAT_or_SCALE='LAT_UNI',  # LAT_UNI
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[0],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)
plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs, sel_subs=LID_SUBS,
    LAT_or_SCALE='LAT_UNI',
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[1],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

# equalize axes
ymin = min([min(ax.get_ylim()) for ax in axes])
ymax = max([max(ax.get_ylim()) for ax in axes])
for ax in axes: ax.set_ylim(ymin, ymax)

for ax in axes: ax.tick_params(axis='both', size=fsize,
                               labelsize=fsize)
plt.tight_layout()

if SAVE_PLOT:
    plt.savefig(os.path.join(get_project_path('figures'),
                             'ft_exploration',
                             f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                             'descr_PSDs', fig_name),
                facecolor='w', dpi=300,)
if SHOW_PLOT: plt.show()
else: plt.close()